# TP9 : Systèmes de Recommandation avec Autoencodeur  

**Brahim ALAOUI**

**Master : BIBDA**


Ce TP vous guide à travers l'ensemble du processus de construction d'un système de recommandation en utilisant un autoencodeur, depuis la préparation des données jusqu'à la génération des recommandations.

Appliquer le modèle Autoencoder sur la dataset Jester.
* Elle contient des évaluations de **blagues** par des utilisateurs.


## PARTIES 1

**1.	Importation des Bibliothèques / Chargement et Préparation des Données:**

* Importer les bibliothèques nécessaires. 
* Télécharger le dataset Jester.
* Charger les données avec pandas et afficher quelques lignes pour s'assurer que les données sont correctement chargées.


In [67]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [68]:
# Charger les données
df_ratings = pd.read_csv('jester/jester_ratings.csv')
df_istem = pd.read_csv('jester/jester_items.csv')
# Afficher les premières lignes des données
print(df_ratings.head())
print("//////////////")
print(df_items.head())

   userId  jokeId  rating
0       1       5   0.219
1       1       7  -9.281
2       1       8  -9.281
3       1      13  -6.781
4       1      15   0.875
//////////////
   jokeId                                           jokeText
0       1  A man visits the doctor. The doctor says "I ha...
1       2  This couple had an excellent relationship goin...
2       3  Q. What's 200 feet long and has 4 teeth? \n\nA...
3       4  Q. What's the difference between a man and a t...
4       5  Q.\tWhat's O. J. Simpson's Internet address? \...


## PARTIES 2

**2.	Prétraitement des Données :**

* Créer une matrice utilisateur-joke.
* Diviser les données en ensembles d'entraînement et de test.
* Convertir les ensembles en matrices.

In [72]:
# Chaque ligne représente un utilisateur et chaque colonne représente joke
# Les valeurs sont les notes que les utilisateurs ont attribuées aux jokes
user_joke_matrix = df_ratings.pivot(index='userId', columns='jokeId', values='rating').fillna(0)


# Diviser les données en ensembles d'entraînement et de test
# En utilisant 80% des données pour l'entraînement et 20% pour le test
train_data, test_data = train_test_split(user_joke_matrix, test_size=0.2, random_state=42)


# Convertir les ensembles d'entraînement et de test en matrices
train_data_matrix = train_data.values
test_data_matrix = test_data.values

## PARTIES 3

**3.	Construction du Modèle Autoencodeur :**

* Définir l'architecture de l'autoencodeur.
* Spécifier les couches d'entrée, encodées et décodées. 
* Compiler le modèle en utilisant l'optimiseur Adam et une fonction de perte appropriée.


In [76]:
# Définition de l'Autoencodeur
# input_dim est le nombre de jokes (le nombre de colonnes dans user_joke_matrix)
# encoding_dim est la dimension de la couche cachée, choisie comme 64 pour réduire la dimensionnalité tout en capturant les informations essentielles
input_dim = train_data_matrix.shape[1]
encoding_dim = 64  # Nombre de neurones dans la couche encodée

# Couche d'entrée
input_layer = tf.keras.layers.Input(shape=(input_dim,))

# Couche encodée avec activation ReLU
# ReLU (Rectified Linear Unit) est utilisée ici car elle aide à résoudre le problème de la vanishing gradient et accélère la convergence
encoded = tf.keras.layers.Dense(encoding_dim, activation='relu')(input_layer)

# Couche décodée avec activation Sigmoid
# Sigmoid est utilisée pour ramener les valeurs dans l'intervalle [0, 1], ce qui est approprié car les notes des jokes sont normalisées
decoded = tf.keras.layers.Dense(input_dim, activation='sigmoid')(encoded)

# Modèle autoencodeur
autoencoder = tf.keras.Model(inputs=input_layer, outputs=decoded)

In [77]:
# Compilation du modèle
# Adam est un optimiseur adaptatif efficace qui fonctionne bien avec les grands datasets et les réseaux de neurones
# La fonction de perte est la Mean Squared Error (MSE) car nous cherchons à minimiser la différence quadratique entre les notes prédites et réelles
autoencoder.compile(optimizer='adam', loss='mean_squared_error')

## PARTIES 4

**4.	Entraînement du Modèle:**

* Entraîner l'autoencodeur sur les données d'entraînement.
1.	En validant le modèle sur les données de test. 
2.	Utiliser un nombre d'époques = 50 et une taille de lot appropriés = 256.


In [81]:
# epochs: nombre de fois que le modèle va voir l'ensemble des données d'entraînement
# batch_size: nombre de samples que le modèle va voir avant d'ajuster les poids
# shuffle=True permet de mélanger les données d'entraînement pour garantir que le modèle ne mémorise pas l'ordre des données
autoencoder.fit(train_data_matrix, train_data_matrix,
                epochs=50,
                batch_size=256,
                shuffle=True,
                validation_data=(test_data_matrix, test_data_matrix))

Epoch 1/50
185/185 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - loss: 6.1007 - val_loss: 5.6402
Epoch 2/50
185/185 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - loss: 5.6944 - val_loss: 5.5499
Epoch 3/50
185/185 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - loss: 5.5236 - val_loss: 5.5102
Epoch 4/50
185/185 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 5.5461 - val_loss: 5.4879
Epoch 5/50
185/185 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - loss: 5.5369 - val_loss: 5.4732
Epoch 6/50
185/185 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - loss: 5.4724 - val_loss: 5.4635
Epoch 7/50
185/185 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - loss: 5.4527 - val_loss: 5.4563
Epoch 8/50
185/185 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 5.4558 - val_loss: 5.4517
Epoch 9/50
185/185 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 5.4174 - val_loss: 5.4479
Epoch 10/50
185/185 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 5.4634 - val_loss: 5.4450
Epoch 11/50
185/185 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - loss: 5.5242 - val_loss: 5.4422
Epoch 12/50
185/185 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/ste

## PARTIES 5

**5.	Évaluation du Modèle:**

* Évaluer les performances de l'autoencodeur en calculant la perte sur les données de test.

In [85]:
loss = autoencoder.evaluate(test_data_matrix, test_data_matrix, verbose=2)
print(f'\nTest Loss: {loss}')


370/370 - 1s - 2ms/step - loss: 5.4044

Test Loss: 5.404417037963867


## PARTIES 6

**6.	Résultats :**

* Utiliser l'encodeur pour transformer les données utilisateur-joke en représentations encodées. * 	Implémenter une fonction pour recommander desjokess non vus par un utilisateur, en se basant sur ces représentations encodées


In [89]:
# Utiliser l'autoencodeur pour obtenir les représentations encodées des jokes
encoded_jokes = autoencoder.predict(test_data_matrix)

370/370 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step


In [90]:
# Fonction de recommandation de blagues
def recommend_jokes(user_id, encoded_jokes, user_joke_matrix, num_recommendations=5):
    # Trouver l'index de l'utilisateur dans la matrice
    user_index = user_joke_matrix.index.get_loc(user_id)
    
    # Obtenir les notes prédites pour cet utilisateur
    user_ratings = encoded_jokes[user_index]
    
    # Trier les blagues par notes décroissantes
    recommendations = np.argsort(user_ratings)[::-1]
    
    # Trouver les blagues non vus par l'utilisateur
    unseen_jokes = user_joke_matrix.columns[user_joke_matrix.loc[user_id] == 0]
    
    # Recommander les blagues non vus avec les scores les plus élevés
    recommended_jokes = unseen_jokes.intersection(recommendations)
    return recommended_jokes[:num_recommendations]

In [91]:
# Exemple de recommandation pour l'utilisateur 1
recommended_jokes = recommend_jokes(1, encoded_jokes, user_joke_matrix)
print(f"Recommended blagues for user 1: {recommended_jokes}")

Recommended blagues for user 1: Index([28, 30, 33, 37, 38], dtype='int64', name='jokeId')


In [92]:
df_istem.head()

,jokeId,jokeText
0,1,"A man visits the doctor. The doctor says ""I ha..."
1,2,This couple had an excellent relationship goin...
2,3,Q. What's 200 feet long and has 4 teeth? \n\nA...
3,4,Q. What's the difference between a man and a t...
4,5,Q.\tWhat's O. J. Simpson's Internet address? \...


In [93]:
# Fonction pour récupérer le text des blagues à partir d'une liste d'IDs de blagues
def get_joke_details(jokes_ids, df_istem):
    details = df_istem[df_istem['jokeId'].isin(jokes_ids)][['jokeText']]
    return details

In [94]:
# Exemple d'utilisation avec un tableau d'IDs de blagues
joke_details = get_joke_details(recommended_jokes, df_istem)

print(joke_details)

                                             jokeText
27  A mechanical, electrical and a software engine...
29  Q: What's the difference between a Lawyer and ...
32  What do you call an American in the finals of ...
36  A Jewish young man was seeing a psychiatrist f...
37  "May I take your order?" the waiter asked. \n\...
